In [ ]:
NOW = ["output/bas1c", "output/droite1c", "output/gauche1c", "output/haut1c", "output/milieu1c"]
ONE_YEAR_AGO = ["output/bas2018", "output/droite2018", "output/gauche2018", "output/haut2018", "output/milieu2018"]
COUNTAIN_START = "coffee"

# Première étape : charger les données

Importer le nom des données

In [ ]:
import os 
import numpy as np
import random
import rasterio
os.environ['CUDA_VISIBLE_DEVICES']='1'
from keras.models import load_model

In [ ]:
def get_all_images_names(directories): 
    images_name = []
    for directory in directories:
        counter = 0
        for filename in os.listdir(directory): 
            if os.path.isfile(directory + "/" + filename):
                images_name.append(directory + "/" + filename)
                counter += 1
            elif os.path.isdir(directory + "/" + filename):
                images_name += get_all_images_names([directory + "/" + filename])
        print(directory, counter, "images.")
    return images_name

In [ ]:
images_name_now = get_all_images_names(NOW)

In [ ]:
images_name_ago = get_all_images_names(ONE_YEAR_AGO)

Importer les images et leur résultat voulu

In [ ]:
def reshape(image):
    """Reformate les données rasterio pour être utilisée par Tensorflow"""
    reshaped_image = []
    for i in range(len(image[0])):
        reshaped_row = []
        for j in range(len(image[0][0])):
            reshaped_cell = []
            #Automne
            reshaped_cell.append(image[0][i][j] / 65535) #Infra rouge
            reshaped_cell.append(image[1][i][j] / 65535) #Rouge
            reshaped_cell.append(image[2][i][j] / 65535) #Vert
            reshaped_cell.append(image[3][i][j] / 65535) #Bleu
            #Hiver
            reshaped_cell.append(image[4][i][j] / 65535) #Infra rouge
            reshaped_cell.append(image[5][i][j] / 65535) #Rouge
            reshaped_cell.append(image[6][i][j] / 65535) #Vert
            reshaped_cell.append(image[7][i][j] / 65535) #Bleu
            #Printemps
            reshaped_cell.append(image[8][i][j] / 65535) #Infra rouge
            reshaped_cell.append(image[9][i][j] / 65535) #Rouge
            reshaped_cell.append(image[10][i][j] / 65535) #Vert
            reshaped_cell.append(image[11][i][j] / 65535) #Bleu
            
            #Si les images ne possèdent pas l'intégralité des 12 bandes, utilisez plutôt le code suivant
            #for k in range(len(image)):
            #    reshaped_cell.append(image[k][i][j] / 65535)
            
            reshaped_row.append(reshaped_cell)
        reshaped_image.append(reshaped_row)
    return reshaped_image

In [ ]:
def load_images(images_name):
    images = []
    classes = []
    for image_name in images_name:
        images.append(reshape(np.array(rasterio.open(image_name).read())))
        if COUNTAIN_START in image_name:
            classes.append(1)
        else:
            classes.append(0)
    print("Longueur initiale : ", len(images))
    return np.array(images).reshape(len(images), len(images[0]), len(images[0][0]), len(images[0][0][0])), np.array(classes)

In [ ]:
images_now, classes_now = load_images(images_name_now)

In [ ]:
images_ago, classes_ago = load_images(images_name_ago)

In [ ]:
model = load_model('GoodModel/model1c6Bands.h5')
model.summary()

# Deuxième étape : tester

In [ ]:
def get_fscore_recall_precision(truth, prediction):
    true_positive = 0
    false_negative = 0
    false_positive = 0
    for i in range(len(truth)):
        if truth[i] == 1 and prediction[i] == 1:
            true_positive += 1
        elif truth[i] == 0 and prediction[i] == 1:
            false_positive += 1
        elif truth[i] == 1 and prediction[i] == 0:
            false_negative += 1
    precision = true_positive/(true_positive + false_positive) if (true_positive + false_positive) > 0 else None
    recall = true_positive/(true_positive + false_negative) if (true_positive + false_negative) > 0 else None
    fscore = 2 * recall * precision / (recall + precision) if recall != None and precision != None else None
    return fscore, recall, precision

In [ ]:
from sklearn import metrics as me
# evaluate the model now
print("Now")
scores = model.evaluate(images_now, classes_now, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print('Test accuracy:', scores[1])
pred = model.predict_classes(images_now)
f, r, p = get_fscore_recall_precision(classes_now, pred)
print('Test recall', r)
print('Test precision', p)
print('Test f-score', f)
print(me.confusion_matrix(classes_now, pred))

# evaluate the model a year ago
print("A year ago")
scores = model.evaluate(images_ago, classes_ago, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print('Test accuracy:', scores[1])
pred = model.predict_classes(images_ago)
f, r, p = get_fscore_recall_precision(classes_ago, pred)
print('Test recall', r)
print('Test precision', p)
print('Test f-score', f)
print(me.confusion_matrix(classes_ago, pred))
    